In [1]:
#定义路网的邻接矩阵
#从路网的邻接矩阵中发现一次刷卡可达的路径名单
#两次刷卡可达就是需要中转或者跨终点站这种。这种不需要考虑，只用一次刷卡可达的信息就够用了
#通过一次刷卡的信息我们可以统计出各种需求的用户的实际刷卡次数。
#进而识别出不同用户会对各个站点带来多大的影响，


# 先把数据读入

- 读入交通数据，处理为`dataframe`方便后面进一步处理

In [1]:
import sys
print(sys.version,sys.executable)
import pandas as pd
sheet1 = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
#线路编号	方向	站点编号	站点名称	日期	   刷卡数量
#1         1     1        火车站    2023-10-01  17
#print(sheet1.head())

sheet2 = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')
#	id	line_identity	line_direction	station_number	station_identity	station_name	longitude	latitude
#    1      1047               1              55                 1           碧海商业广场     106.618469  26.633052
#print(sheet2.head())

3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0] /home/thu_huph22/miniconda3/envs/sumo/bin/python


In [2]:
#方向有1有0，有双向线路也有单向线路，所以需要进行分析，如果方向只有一个，那么起点就是终点
#如果方向有两个，那么终点需要单独拿进来。
#先按照此站下一站的关系聚个类
#统计一下线路-id-（站点，数量）的字典，构造为字典方便索引
#print(sheet1[sheet1['线路编号']==1])
#线路编号	方向	站点编号	站点名称	    日期	    刷卡数量
#1          1     1        火车站       2023-10-01    17
line_direct_seq_stat_nums={}
for _ in range(len(sheet1)):
    line = sheet1[_:_+1]['线路编号'].values[0]
    sequence = sheet1[_:_+1]['站点编号'].values[0]
    station = sheet1[_:_+1]['站点名称'].values[0]
    nums = sheet1[_:_+1]['刷卡数量'].values[0]
    direction = sheet1[_:_+1]['方向'].values[0]
    if line not in line_direct_seq_stat_nums:
        temp={direction:{sequence:(station,nums)}}
        line_direct_seq_stat_nums[line]=temp
        continue
    if direction not in line_direct_seq_stat_nums[line]:
        temp={sequence:(station,nums)}
        line_direct_seq_stat_nums[line][direction]=temp
        continue
    if sequence not in line_direct_seq_stat_nums[line][direction]:
        line_direct_seq_stat_nums[line][direction][sequence]=(station,nums)
        continue
#构造字典，实现目标，通过索引几号线什么方向第几站即可知道是什么站点

In [3]:
#line_direct_seq_stat_nums
#需要整理为 station：可达station，线路，站点编号
station_arrive_line_sequen={}
for line in line_direct_seq_stat_nums.keys():
    direction_list = list(line_direct_seq_stat_nums[line].keys())
    direction_list.sort()
    if len(direction_list)==1:
        direction=direction_list[0]
        sequen = list(line_direct_seq_stat_nums[line][direction].keys())
        sequen.sort()
        for index in range(len(sequen)):
            start_station = line_direct_seq_stat_nums[line][direction][sequen[index]][0]
            if index==0:
                temp_range=0
            else:
                temp_range=1
            for target_index in range(index+1,len(sequen)+temp_range):
                target_station = line_direct_seq_stat_nums[line][direction][sequen[target_index%len(sequen)]][0]
                if start_station not in station_arrive_line_sequen:
                    temp={target_station:[(line,sequen[target_index%len(sequen)])]}
                    station_arrive_line_sequen[start_station]=temp
                    continue
                if target_station not in station_arrive_line_sequen[start_station]:
                    temp=[(line,sequen[target_index%len(sequen)])]
                    station_arrive_line_sequen[start_station][target_station]=temp
                    continue
                temp=(line,sequen[target_index%len(sequen)])
                station_arrive_line_sequen[start_station][target_station].append(temp)
    else:
        for direction_index in range(len(direction_list)):
            direction = direction_list[direction_index]
            inverse_direction = direction_list[(direction_index+1)%len(direction_list)]
            sequen_direction = list(line_direct_seq_stat_nums[line][direction].keys())
            sequen_direction.sort()
            sequen_inverse_direction = list(line_direct_seq_stat_nums[line][inverse_direction].keys())
            sequen_inverse_direction.sort()
            end_station,end_nums = line_direct_seq_stat_nums[line][inverse_direction][sequen_inverse_direction[0]]
            for index in range(len(sequen_direction)):
                start_station= line_direct_seq_stat_nums[line][direction][sequen_direction[index]][0]
                for target_index in range(index+1,len(sequen_direction)):
                    target_station = line_direct_seq_stat_nums[line][direction][sequen_direction[target_index]][0]
                    if start_station not in station_arrive_line_sequen:
                        temp={target_station:[(line,sequen_direction[target_index])]}
                        station_arrive_line_sequen[start_station]=temp
                        continue
                    if target_station not in station_arrive_line_sequen[start_station]:
                        temp=[(line,sequen_direction[target_index])]
                        station_arrive_line_sequen[start_station][target_station]=temp
                        continue
                    temp=(line,sequen_direction[target_index])
                    station_arrive_line_sequen[start_station][target_station].append(temp)
                #print(start_station,end_station)
                if start_station not in station_arrive_line_sequen:
                    temp={end_station:[(line,sequen_inverse_direction[0])]}
                    station_arrive_line_sequen[start_station]=temp
                    continue
                if end_station not in station_arrive_line_sequen[start_station]:
                    temp=[(line,sequen_inverse_direction[0])]
                    station_arrive_line_sequen[start_station][end_station]=temp
                    continue
                temp=(line,sequen_inverse_direction[0])
                station_arrive_line_sequen[start_station][end_station].append(temp)
print("dataframe done")
#从数据中可以看出，一共有69164个od对是一次刷卡即可达到的，不用两次刷卡。
#根据我们的假设，出发必须要回来，我们可以找出所有的可行路径。
#站点之间的一次可达对一共有49443，考虑到同一个站点对不同线路，则有69164种，
#对于每个站点，平均可达站点有：43.44，考虑到线路就有：60种可能，遍历下来可能要很久很久。如果我们及时止损，比如说从出发到回家不能转车超过6次，比如说去转车三次，回来三次。

dataframe done


In [10]:
#定义一个函数，实现从起点开始找回到起点的路径。
#_station_path={站点：【可行方案列表】}
_station_path={}
for _station in station_arrive_line_sequen.keys():
    node_path=[]
    for station_ in station_arrive_line_sequen[_station].keys():
        for way in station_arrive_line_sequen[_station][station_]:
            line,node_id=way
            node_path.append(station_+'_'+str(line)+'_'+str(node_id))
    _station_path[_station]=node_path

In [11]:
_station_path['黔灵山公园']

['延安西路_1_13',
 '延安西路_41_30',
 '延安西路_238_9',
 '紫林庵_1_14',
 '紫林庵_16_18',
 '紫林庵_23_33',
 '紫林庵_41_31',
 '紫林庵_238_10',
 '大西门_1_15',
 '大西门_16_19',
 '大西门_23_34',
 '大西门_41_32',
 '次南门_1_16',
 '次南门_16_20',
 '次南门_22_35',
 '河滨公园_1_17',
 '河滨公园_16_21',
 '河滨公园_23_35',
 '新路口_1_18',
 '新路口_2_18',
 '新路口_23_36',
 '展览馆_1_19',
 '展览馆_2_19',
 '火车站_1_1',
 '火车站_2_1',
 '北京西路口_2_11',
 '北京西路口_12_13',
 '北京西路口_16_15',
 '北京西路口_23_30',
 '北京西路口_27_13',
 '北京西路口_33_27',
 '北京西路口_72_28',
 '北京西路口_77_24',
 '北京西路口_248_34',
 '北京西路口_261_17',
 '北京西路口_311_12',
 '北京西路口_314_30',
 '北京西路口_316_22',
 '北京西路口_320_13',
 '北京路_2_12',
 '北京路_12_14',
 '北京路_22_25',
 '北京路_27_14',
 '北京路_33_28',
 '北京路_235_4',
 '北京路_248_35',
 '北京路_261_18',
 '北京路_311_13',
 '北京路_314_31',
 '北京路_316_23',
 '北京路_320_15',
 '六广门_2_13',
 '云中广场_2_14',
 '喷水池_2_15',
 '喷水池_238_11',
 '大十字_2_16',
 '大十字_22_32',
 '邮电大楼_2_17',
 '浣沙桥_10_16',
 '浣沙桥_33_12',
 '浣沙桥_53_30',
 '浣沙桥_72_12',
 '浣沙桥_248_45',
 '花香村_10_17',
 '花香村_33_13',
 '花香村_53_31',
 '花香村_72_13',
 '花香村_248_46',
 '体育馆_10_18',
 '

In [13]:
import copy
#从上面的数据中找出所有可能的起点与终点相同的路径，设置一个最大路径长，也就是刷卡6次之内可以完成的操作。
path_max_length=6
#all_possible_path={站点：[路径列表]}
all_possible_path={}
#path:字典{线路：【经过的站点id列表】}
#由于我们的目标是找到所有可行的路径，所以需要设置一个visited和unvisted 节点表。
def find_all_path(start_node,end_node,_station_path,paths,path_max_length):
    #path应该是一条路径的情况，不能把所有的情况输入，应该方便回溯。已有的路径看是不是构成了一个环
    paths=copy.deepcopy(paths)
    if path_max_length==1:
        all_paths=[]
        for path in paths:
            for way in _station_path[start_node]:
                station,line,in_line_id=way.split('_')
                if station==end_node:
                    path.append(way)
                    all_paths.append(copy.deepcopy(path))
                    path.pop()
        return all_paths
    elif path_max_length==0:
        all_paths=[]
        return all_paths
    else:
        all_paths=[]
        for way in _station_path[start_node]:
            for path in paths:
                path.append(way)
            #采取动作为剩余动作空间中的第一个动作
            station,line,in_line_id=way.split('_')
            temp_path = find_all_path(station,end_node,_station_path,paths,path_max_length-1)
            if len(temp_path)>0:
                all_paths = all_paths+copy.deepcopy(temp_path)
            for path in paths:
                path.pop()
        return all_paths
#定义一个函数，可以讲路径列表，转化为字典：
def transformer(path):
    out_dict={}
    for way in path:
        station,line,in_line_id=way.split('_')
        if line not in out_dict:
            out_dict[line]=[in_line_id]
            continue
        if in_line_id not in out_dict[line]:
            out_dict[line].append(in_line_id)
            continue
    return out_dict

In [5]:
sum=0
for station in station_arrive_line_sequen:
    #for arrive in station_arrive_line_sequen[station]:
        #sum +=1
    sum +=1
print(sum)

1138


In [4]:
sum=0
for station in station_arrive_line_sequen:
    for arrive in station_arrive_line_sequen[station]:
        sum +=len(station_arrive_line_sequen[station][arrive])
print(sum)

69164


In [53]:
output = find_all_path('黔灵山公园','喷水池',_station_path,[['黔灵山公园']],2)

In [54]:
output

[['黔灵山公园', '延安西路_1_13', '喷水池_3_13'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_19_11'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_28_30'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_34_9'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_45_6'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_54_38'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_73_10'],
 ['黔灵山公园', '延安西路_1_13', '喷水池_238_11'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_3_13'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_19_11'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_28_30'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_34_9'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_45_6'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_54_38'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_73_10'],
 ['黔灵山公园', '延安西路_41_30', '喷水池_238_11'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_3_13'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_19_11'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_28_30'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_34_9'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_45_6'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_54_38'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_73_10'],
 ['黔灵山公园', '延安西路_238_9', '喷水池_238_11'],
 ['黔灵山公园', '紫林庵_1_14', '喷水池_3_13'],
 ['黔灵山公园', '紫林庵_1_14', '喷水池_28_30'],
 ['黔灵

In [14]:
output = find_all_path('黔灵山公园','黔灵山公园',_station_path,[['黔灵山公园']],2)

In [ ]:
sheet1[sheet1['线路编号']==1]
station_tt=sheet1.groupby('线路编号').agg({'方向':set}).reset_index()

In [ ]:
station_tt[station_tt['方向']==set([1])]
#有191条线路是标记为双向的，
#有11条线路标记为单向1
#有1条线路标记为单向0

In [21]:
station_arrive_line_sequen['邮电大楼']

{'大十字': [(1, 5)],
 '喷水池': [(1, 6)],
 '云中广场': [(1, 7)],
 '六广门': [(1, 8)],
 '北京路': [(1, 9), (15, 46)],
 '北京西路口': [(1, 10), (218, 28)],
 '黔灵山公园': [(1, 12)],
 '新路口': [(2, 18), (15, 40), (29, 37), (253, 25), (272, 12), (308, 14)],
 '展览馆': [(2, 19), (29, 38), (253, 26), (253, 29), (308, 15)],
 '火车站': [(2, 1), (253, 27), (272, 15)],
 '河滨公园': [(15, 41), (218, 25)],
 '次南门': [(15, 42)],
 '大西门': [(15, 43)],
 '紫林庵': [(15, 44), (218, 27)],
 '威清路口': [(15, 45)],
 '市北路': [(15, 47)],
 '扁井': [(15, 48)],
 '南馨苑': [(15, 50)],
 '南垭路口': [(15, 51)],
 '鹿冲关公园': [(15, 2)],
 '甲秀楼': [(15, 17), (52, 10), (307, 14)],
 '省委': [(15, 18), (52, 11), (307, 15), (307, 16)],
 '观水路': [(15, 19)],
 '观水路口': [(15, 20)],
 '水口寺': [(15, 21)],
 '水口寺大桥': [(15, 22), (306, 21)],
 '红岩路北岸': [(15, 23)],
 '红岩桥': [(15, 24), (306, 23)],
 '阿玛施眼科': [(15, 26)],
 '新城路': [(15, 27)],
 '王家桥': [(15, 29)],
 '市一医': [(20, 9)],
 '市府路口': [(20, 10)],
 '人民大道中': [(20, 11)],
 '黔灵西路口': [(20, 12)],
 '人民大道北': [(20, 13)],
 '渔安隧道': [(20, 15)],
 '大营路': [(20, 16)],

In [26]:
sheet1[sheet1['线路编号']==2]
#如果连接序列中有比他小的也有比他大的，并且节点编号上来看节点小的在节点大的之前，那么就会导致多转圈。

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
18,2,1,1,火车站,2023-10-01,22
19,2,1,2,展览馆,2023-10-01,232
20,2,1,3,新路口,2023-10-01,486
21,2,1,4,河滨公园,2023-10-01,85
22,2,1,5,次南门,2023-10-01,146
23,2,1,6,大西门,2023-10-01,209
24,2,1,7,紫林庵,2023-10-01,53
25,2,1,8,延安西路,2023-10-01,327
26,2,0,10,黔灵山公园,2023-10-01,244
27,2,0,11,北京西路口,2023-10-01,860


In [19]:
station_arrive_line_sequen['友邻路']

{'友邻北路': [(316, 2)],
 '未来方舟': [(316, 3)],
 '博学北路': [(316, 4)],
 '云中区政务中心': [(316, 5)],
 '渔安立交': [(316, 6)],
 '渔安隧道': [(316, 7)],
 '贵州医科大学': [(316, 8)],
 '北京路': [(316, 9)],
 '北京西路口': [(316, 10)],
 '黔灵山公园': [(316, 11)],
 '省肿瘤医院': [(316, 12)],
 '圣泉流云花园': [(316, 15)]}

In [8]:
sheet1[sheet1['线路编号']==1]
station_tt=sheet1.groupby('线路编号').agg({'方向':set}).reset_index()

In [9]:
station_tt[station_tt['方向']==set([1])]
#有191条线路是标记为双向的，
#有11条线路标记为单向1
#有1条线路标记为单向0

,线路编号,方向
87,211,{1}
100,228,{1}
136,280,{1}
144,305,{1}
172,606,{1}
173,607,{1}
178,616,{1}
186,804,{1}
191,809,{1}
201,4600,{1}


In [20]:
sheet1[sheet1['线路编号']==316]

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
5560,316,1,1,友邻路,2023-10-01,5
5561,316,1,2,友邻北路,2023-10-01,117
5562,316,1,3,未来方舟,2023-10-01,121
5563,316,1,4,博学北路,2023-10-01,136
5564,316,1,5,云中区政务中心,2023-10-01,119
5565,316,1,6,渔安立交,2023-10-01,99
5566,316,1,7,渔安隧道,2023-10-01,158
5567,316,1,8,贵州医科大学,2023-10-01,76
5568,316,1,9,北京路,2023-10-01,53
5569,316,1,10,北京西路口,2023-10-01,92


In [4]:
#先按照此站下一站的关系聚个类
#统计一下线路-id-（站点，数量）的字典，构造为字典方便索引
#print(sheet1[sheet1['线路编号']==1])
#线路编号	方向	站点编号	站点名称	    日期	    刷卡数量
#1          1     1        火车站       2023-10-01    17
line_seq_stat_nums={}
for _ in range(len(sheet1)):
    line = sheet1[_:_+1]['线路编号'].values[0]
    sequence = sheet1[_:_+1]['站点编号'].values[0]
    station = sheet1[_:_+1]['站点名称'].values[0]
    nums = sheet1[_:_+1]['刷卡数量'].values[0]
    direction = sheet1[_:_+1]['方向'].values[0]
    if line not in line_seq_stat_nums:
        temp={sequence:(station,nums,direction)}
        line_seq_stat_nums[line]=temp
        continue
    if sequence not in line_seq_stat_nums[line]:
        line_seq_stat_nums[line][sequence]=(station,nums,direction)
        continue


In [7]:
station_arrive_line={}
for line in line_seq_stat_nums.keys():
    sequen = list(line_seq_stat_nums[line].keys())
    sequen.sort()
    for index in range(len(sequen)):
        this_station=line_seq_stat_nums[line][sequen[index]][0]
        if index==0:
            temp_range=0
        else:
            temp_range=1
        for target_index in range(index+1,len(sequen)+temp_range):
            target_station = line_seq_stat_nums[line][sequen[target_index%len(sequen)]][0]
            if this_station not in station_arrive_line:
                temp={target_station:[(line,sequen[target_index%len(sequen)])]}
                station_arrive_line[this_station]=temp
            else:
                if target_station not in station_arrive_line[this_station]:
                    temp=[(line,sequen[target_index%len(sequen)])]
                    station_arrive_line[this_station][target_station]=temp
                else:
                    temp=(line,sequen[target_index%len(sequen)])
                    station_arrive_line[this_station][target_station].append(temp)
print("dataframe done")
#此站          下站    线路   上车数量  此站编号 下站编号 单程终点
#火车站       展览馆     1      17        1       2      1
#展览馆       新路口     1      318       2       3      1
#北京西路口  黔灵山公园   1      582       10      12      0
#print(this_next_line_nums[this_next_line_nums['线路']==1])

dataframe done


In [9]:
sum=0
for station in station_arrive_line:
    for arrive in station_arrive_line[station]:
        sum +=len(station_arrive_line[station][arrive])
print(sum)

132275


# 统计公交线路的基本信息

- 观察数据结构
- 识别出终点和中转站点
1. 不同站点经过的线路数是不一样的，所有站点累计经过6480条线路，每个站点平均经过线路数为6条。来回也算的话，大概三条
2. 暂无

In [2]:
station_line=sheet1.groupby('站点名称').agg({'线路编号':list}).reset_index()
#站点名称       线路编号
#(招呼站)	  [226, 226, 228]
#station_line.head()
#print(len(station_line))#1138
#实际经过了1138个站点，其中还有三个是站点表里面没有的。站点表里有1449个站点，加上这三个不在的站点。一共有1452个站点，
station_line['lines'] = station_line.apply(lambda x : len(x['线路编号']),axis=1)
#站点名称         线路编号           lines
#(招呼站)	  [226, 226, 228]          3
#print(sum(station_line['lines']))#6480

- 把所有的站点统计为字典，分为名字-id和id-名字两个映射
- 统计一下各条线路经过的站点量
1. 每条线路平均经过的站点数为：31.92.（因为贵阳的公交往返路线很多是不一致的，所以从起点到起点这个圈上所有经过的站点）
2. 线路总数为203条，线路经过的站点总数为6480，平均每条线路的站点数为：31.92站。（一圈）

In [3]:
#处理已知站点，不用地理位置信息，只根据流入流出确定
temp_station=sheet2.groupby('station_name').agg({'id':list}).reset_index()
# station_name                           id
#  (招呼站)             [7304, 7308, 7317, 7321, 10728, 10760]
#print(temp_station.head())
#把已知的站点信息统计一下
station_id={}
#station_name : id 
#(招呼站)     -> 0
for i in range(len(temp_station)):
    station_id[temp_station[i:i+1]['station_name'].values[0]]=i

print("station to id done")
#for key in station_id.keys():
#    print(key,'->',station_id[key])
#    break
id_station={}
#id -> station
#0 -> (招呼站)
for key,value in station_id.items():
    id_station[value]=key
print("id to station done")
#for key in id_station.keys():
#    print(key,'->',id_station[key])
#    break

#输出已知站点总数
#print(len(temp_station))#1449

line_station=sheet1.groupby('线路编号').agg({'站点名称':list}).reset_index()
#线路编号                        站点名称
#1         [火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...
#print(line_station.head())
line_station['sites']=line_station.apply(lambda x:len(x['站点名称']),axis=1)
#线路编号                               站点名称                                       站点量
#1          [火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...     18
#print(line_station.head())
#print(len(line_station))#203
#给每条线路上的每个站点一个编号
#print(sum(temp_line['sites']))#6480

station to id done
id to station done


- 把站点字典扩充一下，把实际节点中有的但是节点列表中没有的添加进去。

1. 实际线路一共有203条，实际节点1138个，站点列表中一共有1449个，其中有三个没涵盖。
2. 从线路上看总站点数为6480，而实际站点数只有1138.所以很多线路会有交叉，平均每个站点重用了6次，平均每条线路有31.92个站点。
3. 添加后的节点数为1452

In [4]:
#补充站点列表不包含的站点编号
not_contain= sheet1[~ sheet1['站点名称'].isin(station_id.keys())]
not_contain_station = not_contain.groupby('站点名称').agg({'刷卡数量':list}).reset_index()
for i in range(len(not_contain_station)):
    id_station[len(station_id)]=not_contain_station[i:i+1]['站点名称'].values[0]
    station_id[not_contain_station[i:i+1]['站点名称'].values[0]]=len(station_id)
#print(len(id_station))#1452

- 根据我们要构造的图的形式：节点和边（节点表示站点，同时由于两个站点之间可能有多条线路，为了区分线路，所以加上虚拟的线路站点。）
- 把公交的线路数据构造称为此站点，下站点，上车人数的形式。
- 然后对于每条此站点，下站点数据，做三项工作：1.向节点表中添加新的虚拟中间站点；2.同时创建邻接矩阵并添加两项，一项是起点到中转节点，另一项是中转到目标节点
- 3.在节点列表扩充完毕后，我们需要设计节点的特征，就用在各个节点的上车数据来表示：
- 边用节点特征表示，走什么线路，所以是稀疏矩阵不假，但是有些太大了，不过稀疏矩阵应该也好表示

In [35]:
#字典结构:{线路编号：{方向:{站点编号：(站点名称，刷卡数量)}}}
print("dict done")
#形成一个新的dataframe，结构为：[此站，下站，线路，刷卡数]
#可以使用join来构造，但是数据有问题，站点编号是不连续的，只能一行一行读取
this_next_line_nums=pd.DataFrame(columns=['此站','下站','线路','上车数量','此站编号','下站编号','单程起点'])
for line in line_seq_stat_nums.keys():
    direction_list = list(line_seq_stat_nums[line].keys())
    direction_list.sort()
    for direction in direction_list:
        sequen = list(line_seq_stat_nums[line][direction].keys())
        sequen.sort()
        inverse_direction=(direction+1)%len(direction_list)
        print(line)
        print(inverse_direction)
        inverse_sequen = list(line_seq_stat_nums[line][inverse_direction].keys())
        inverse_sequen.sort()
        final_end_station=line_seq_stat_nums[line][inverse_direction][inverse_sequen[0]][0]
        for index in range(len(sequen)):
            if index != len(sequen)-1:
                ll = {'此站':line_seq_stat_nums[line][direction][sequen[index]][0],'下站':line_seq_stat_nums[line][direction][sequen[(index+1)%len(sequen)]][0],'线路':line,'上车数量':line_seq_stat_nums[line][direction][sequen[index]][1],'此站编号':sequen[index],'下站编号':sequen[(index+1)%len(sequen)],'单程起点':0 if index==0 else 1}
                ##print(ll)
                #this_next_line_nums.append(ll,ignore_index=True)
                this_next_line_nums.loc[len(this_next_line_nums)]=ll
            else:
                ll = {'此站':line_seq_stat_nums[line][direction][sequen[index]][0],'下站':final_end_station,'线路':line,'上车数量':line_seq_stat_nums[line][direction][sequen[index]][1],'此站编号':sequen[index],'下站编号':inverse_sequen[0],'单程起点':1}
                this_next_line_nums.loc[len(this_next_line_nums)]=ll

print("dataframe done")
#此站          下站    线路   上车数量  此站编号 下站编号 单程终点  单程终点
#火车站       展览馆     1      17        1       2      1       0
#展览馆       新路口     1      318       2       3      1       0
#北京西路口  黔灵山公园   1      582       10      12      0       0
#print(this_next_line_nums[this_next_line_nums['线路']==1])

dict done
1
1
1
0
2
1
2
0
3
1
3
0
4
1
4
0
5
1
5
0
6
1
6
0
7
1
7
0
8
1
8
0
9
1
9
0
10
1
10
0
11
1
11
0
12
1
12
0
13
1
13
0
14
1
14
0
15
1
15
0
16
1
16
0
17
1
17
0
18
1
18
0
19
1
19
0
20
1
20
0
21
1
21
0
22
1
22
0
23
1
23
0
25
1
25
0
26
1
26
0
27
1
27
0
28
1
28
0
29
1
29
0
32
1
32
0
33
1
33
0
34
1
34
0
35
1
35
0
36
1
36
0
37
1
37
0
38
1
38
0
39
1
39
0
40
1
40
0
41
1
41
0
42
1
42
0
43
1
43
0
45
1
45
0
46
1
46
0
47
1
47
0
48
1
48
0
49
1
49
0
50
1
50
0
51
1
51
0
52
1
52
0
53
1
53
0
54
1
54
0
55
1
55
0
56
1
56
0
57
1
57
0
58
1
58
0
59
1
59
0
60
1
60
0
61
1
61
0
63
1
63
0
64
1
64
0
65
1
65
0
66
1
66
0
67
1
67
0
69
1
69
0
70
1
70
0
71
1
71
0
72
1
72
0
73
1
73
0
74
1
74
0
75
1
75
0
76
1
76
0
77
1
77
0
78
1
78
0
79
1
79
0
81
1
81
0
82
1
82
0
83
1
83
0
84
1
84
0
86
1
86
0
87
1
87
0
202
1
202
0
203
1
203
0
204
1
204
0
205
1
205
0
206
1
206
0
207
1
207
0
208
1
208
0
210
1
210
0
211
0


KeyError: 0

In [ ]:
#字典结构:{线路编号：{站点编号：(站点名称，刷卡数量，方向)}}
print("dict done")
#形成一个新的dataframe，结构为：[此站，下站，线路，刷卡数]
#可以使用join来构造，但是数据有问题，站点编号是不连续的，只能一行一行读取
this_next_line_nums=pd.DataFrame(columns=['此站','下站','线路','上车数量','此站编号','下站编号','单程终点','单程起点'])
for line in line_seq_stat_nums.keys():
    sequen = list(line_seq_stat_nums[line].keys())
    sequen.sort()
    for index in range(len(sequen)):
        ll = {'此站':line_seq_stat_nums[line][sequen[index]][0],'下站':line_seq_stat_nums[line][sequen[(index+1)%len(sequen)]][0],'线路':line,'上车数量':line_seq_stat_nums[line][sequen[index]][1],'此站编号':sequen[index],'下站编号':sequen[(index+1)%len(sequen)],'单程终点':0 if line_seq_stat_nums[line][sequen[(index-1)%len(sequen)]][2]!=line_seq_stat_nums[line][sequen[index]][2] else 1,'单程起点':0 if (index==0 or line_seq_stat_nums[line][sequen[(index-1)%len(sequen)]][2]!=line_seq_stat_nums[line][sequen[index]][2]) else 1}
        ##print(ll)
        #this_next_line_nums.append(ll,ignore_index=True)
        this_next_line_nums.loc[len(this_next_line_nums)]=ll
print("dataframe done")
#此站          下站    线路   上车数量  此站编号 下站编号 单程终点  单程终点
#火车站       展览馆     1      17        1       2      1       0
#展览馆       新路口     1      318       2       3      1       0
#北京西路口  黔灵山公园   1      582       10      12      0       0
#print(this_next_line_nums[this_next_line_nums['线路']==1])

In [12]:
#定义一票可达的站点
sheet1[sheet1['线路编号']==214]

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
3332,214,1,2,市北路,2023-10-01,16
3333,214,1,3,扁井,2023-10-01,43
3334,214,1,5,保利云山,2023-10-01,16
3335,214,1,7,雅关,2023-10-01,19
3336,214,1,8,雅关大寨,2023-10-01,1
3337,214,1,12,瓦窑寨路口,2023-10-01,2
3338,214,1,13,瓦窑寨,2023-10-01,2
3339,214,1,14,盐沙大道中段,2023-10-01,1
3340,214,1,15,奔土路口,2023-10-01,1
3341,214,1,16,都拉营,2023-10-01,6


In [24]:
end_station = this_next_line_nums[['线路','此站编号','此站','单程终点']]

In [28]:
station_end={}
for _ in range(len(end_station)):
    line = end_station[_:_+1]['线路'].values[0]
    sequence = end_station[_:_+1]['此站编号'].values[0]
    station = end_station[_:_+1]['此站'].values[0]
    flag = end_station[_:_+1]['单程终点'].values[0]
    if line not in station_end:
        temp={sequence:{station:flag}}
        station_end[line]=temp
        continue
    if sequence not in station_end[line]:
        station_end[line][sequence]={station:flag}
        continue

In [30]:
# 生成一次票可达的站点对
this_next_line_nums=pd.DataFrame(columns=['此站','一次可达站点','线路'])
for line in line_seq_stat_nums.keys():
    sequen = list(line_seq_stat_nums[line].keys())
    sequen.sort()
    for index in range(len(sequen)):
        ll = {'此站':line_seq_stat_nums[line][sequen[index]][0],'下站':line_seq_stat_nums[line][sequen[(index+1)%len(sequen)]][0],'线路':line,'上车数量':line_seq_stat_nums[line][sequen[index]][1],'此站编号':sequen[index],'下站编号':sequen[(index+1)%len(sequen)],'单程终点':0 if line_seq_stat_nums[line][sequen[(index-1)%len(sequen)]][2]!=line_seq_stat_nums[line][sequen[index]][2] else 1,'单程起点':0 if (index==0 or line_seq_stat_nums[line][sequen[(index-1)%len(sequen)]][2]!=line_seq_stat_nums[line][sequen[index]][2]) else 1}
        ##print(ll)
        #this_next_line_nums.append(ll,ignore_index=True)
        this_next_line_nums.loc[len(this_next_line_nums)]=ll
print("dataframe done")

{1: {1: {'火车站': 0},
  2: {'展览馆': 1},
  3: {'新路口': 1},
  4: {'邮电大楼': 1},
  5: {'大十字': 1},
  6: {'喷水池': 1},
  7: {'云中广场': 1},
  8: {'六广门': 1},
  9: {'北京路': 1},
  10: {'北京西路口': 1},
  12: {'黔灵山公园': 0},
  13: {'延安西路': 1},
  14: {'紫林庵': 1},
  15: {'大西门': 1},
  16: {'次南门': 1},
  17: {'河滨公园': 1},
  18: {'新路口': 1},
  19: {'展览馆': 1}},
 2: {1: {'火车站': 0},
  2: {'展览馆': 1},
  3: {'新路口': 1},
  4: {'河滨公园': 1},
  5: {'次南门': 1},
  6: {'大西门': 1},
  7: {'紫林庵': 1},
  8: {'延安西路': 1},
  10: {'黔灵山公园': 0},
  11: {'北京西路口': 1},
  12: {'北京路': 1},
  13: {'六广门': 1},
  14: {'云中广场': 1},
  15: {'喷水池': 1},
  16: {'大十字': 1},
  17: {'邮电大楼': 1},
  18: {'新路口': 1},
  19: {'展览馆': 1}},
 3: {1: {'贵州理工学院': 0},
  2: {'贵黄交警支队': 1},
  3: {'枫丹白鹭花园': 1},
  4: {'后坝': 1},
  5: {'三桥水果市场': 1},
  6: {'三桥': 1},
  7: {'三桥北路口': 1},
  8: {'二桥': 1},
  9: {'转弯塘': 1},
  10: {'头桥': 1},
  11: {'延安西路': 1},
  12: {'紫林庵': 1},
  13: {'喷水池': 1},
  15: {'师大': 0},
  16: {'喷水池': 1},
  17: {'黔灵西路口': 1},
  18: {'威清路口': 1},
  19: {'威清路': 1},
  20: {'头桥': 1